# Test Notebook for Building Pipeline Using Module 'fasttext_score'
The module 'fasttext_score' was generated through command: 'az ml module init -n fasttext_score'

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, Pipeline, dsl
import sys
from pathlib import Path
# The following line adds source directory to path. 
from fasttext_score import fasttext_score

## Prerequisites: Configure workspace & compute

 - Update config.json to make sure it has information on your workspace, subscription id, etc.
 - Change the following aml_compute_target's value to reference an existing compute target

In [ ]:
workspace = Workspace.from_config(path = './config.json')
aml_compute_target = "YOUR_COMPUTE_TARGET"
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, aml_compute_target, sep='\n')

# Load module(s)
Load a module function through one of the following:
- Module.from_func: when you have a python function decorated with @dsl.module
- Module.from_yaml: when loading from existing module yaml definition
- Module.load: when you have already registered the module into Azure ML workspace

In [ ]:
module_func = Module.from_func(workspace, fasttext_score)
help(module_func)

# Run module in local
Using module.run can execute module in local machine<br>
If use_docker=True, will pull image from azure and run module in container.<br>
If use_docker=False, will directly run module script.

In [ ]:
# create module
module = module_func().set_parameters(
    param0='abc', param1=10
).set_inputs(
    input_files=str(Path('data') / 'fasttext_score' / 'inputs' / 'input_files'), side_input=str(Path('data') / 'fasttext_score' / 'inputs' / 'side_input')
)
module.run(use_docker=True)

# Create pipeline
This is done by calling module function(s) with the parameters/inputs/outputs the module supports

In [ ]:
# prepare dataset with the following code 

# from azureml.core import Dataset

# your_dataset = Dataset.get_by_name(name='YOUR_DATASET_NAME')


In [ ]:
@dsl.pipeline(name='fasttext_score_pipeline', description='A sample pipeline uses fasttext_score', default_compute_target=aml_compute_target)
def sample_pipeline() -> Pipeline:
    module1 = module_func().set_parameters(
    
    ).set_inputs(
        # Please fill required inputs here
        # input_files=None
    )
    module1.runsettings.configure(node_count=1, process_count_per_node=1)
    return module1.outputs

test_pipeline = sample_pipeline()

In [ ]:
test_pipeline.validate()

In [ ]:
run = test_pipeline.submit(
    experiment_name='fasttext_score_experiment'
)
run.wait_for_completion()
run